#   Práctica 2: Imágenes digitales

## Librerías - Programas

In [ ]:
import cv2
import sys 
import numpy as np
import matplotlib.pyplot as plt
import os.path

def read_pgm_file(file_name):
    """
    Reads a PGM file and returns the image as a grayscale image.

    Args:
        file_name (str): The name of the PGM file to be read.

    Returns:
        numpy.ndarray: The grayscale image read from the PGM file.
    """
    # data_dir = os.path.dirname(os.path.abspath(__file__))

    # # Test if file exists
    # file_path = os.path.join(data_dir, file_name)
    # assert os.path.isfile(file_path), 'file \'{0}\' does not exist'.format(file_path)
    img = cv2.imread(file_name,flags=cv2.IMREAD_GRAYSCALE)

    if img is not None:
        print('img.size: ', img.size)
    else:
        print('imread({0}) -> None'.format(file_name))

    return img

def show_img_hist(im,save=False, Name=""):
    
    vmin = np.amin(im)
    vmax = np.max(im)
    print("Intensity Min: {}   Max:{}".format(vmin,vmax))

    L = np.abs(int(vmax - vmin))
    print("Number of Levels: {}".format(L))
    fig = plt.figure(figsize=(16,6))
    ax1 = plt.subplot(1, 2, 1)
    ax2 = plt.subplot(1, 2, 2)

    # imgplot = plt.imshow(im/np.amax(im))
    imgplot = ax1.imshow(im,cmap='gray', vmin=vmin, vmax=vmax)
    fig.colorbar(imgplot, ax=ax1)
    # cv2.imshow(infile,img)
    # cv2.waitKey(0)

    hist, bin_edges = np.histogram(im.ravel(),bins=L)
    ax2.bar(bin_edges[:-1], hist)
    plt.show()
    if save: 
        fig1 = plt.figure(figsize=(10,10))
        ax1 = fig1.add_subplot(111)
        imgplot = ax1.imshow(im,cmap='gray', vmin=vmin, vmax=vmax)
        fig1.colorbar(imgplot, ax=ax1)
        fig1.savefig(Name+"_img.png", bbox_inches='tight')
        fig2 = plt.figure(figsize=(10,10))
        ax2 = fig2.add_subplot(111)
        ax2.bar(bin_edges[:-1], hist)
        fig2.savefig(Name+"_hist.png", bbox_inches='tight')
    return

## Ex. 1: Histograma de una imagen

### Histograma

In [ ]:
img_A = read_pgm_file('ImagenA.pgm')
show_img_hist(img_A,1,'informe_original')

### Semilineal

In [ ]:
def lineal_transformation(im):
    r0 = np.amin(im)
    r1 = np.amax(im)
    m = 255/(r1-r0)

    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if im[i,j] < r0:
                im[i,j] = 0
            elif im[i,j] > r1:
                im[i,j] = 255
            else:
                im[i,j] = m*(im[i,j]-r0)
    return

lineal_transformation(img_A)
show_img_hist(img_A,1,'informe_semilineal')

## Ex. 2:

### a) Graficar su histograma

img_A = read_pgm_file('ImagenA.pgm')
show_img_hist(img_A)

### b) Ecualizarlo; mostrar la imagen resultante y su nuevo histograma de intensidades

In [ ]:
img_A = read_pgm_file('ImagenA.pgm')
def equalization(im):
    hist, bin_edges = np.histogram(im.ravel(),bins=256) # histograma y bins
    cdf = hist.cumsum() #acumulado
    #   plt.plot(cdf)
    cdf = cdf / cdf[-1] #normalizo
    cdf = 255 * cdf #escalo

    for i in range(im.shape[0]): #recorro la imagen
        for j in range(im.shape[1]): 
            im[i,j] = cdf[im[i,j]] #reemplazo el valor de la imagen por el valor acumulado
    return

equalization(img_A)
show_img_hist(img_A, 1, 'informe_equalizada')

### c) Realizar la transformacio ́n s = T (r) sobre la imagen A:
-   s = 1, 0 < r < 128 ; s = 0, r ≥ 128
-   s=c r^γ con γ>1
-   s=  c r^γ con γ<1

### d) Realizar la substraccion 

In [ ]:
def thresholding(im): 
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if im[i,j] < 128:
                im[i,j] = 255
            else:
                im[i,j] = 0
    return

def gamma_correction(im, gamma):
    transformed_image = im.copy()
    transformed_image = transformed_image.astype(np.float64)
    transformed_image = np.power(transformed_image, gamma)
    #cv2.normalize(transformed_image, transformed_image, 0, 255, cv2.NORM_MINMAX)
    transformed_image = transformed_image *255.0/np.amax(transformed_image)
    transformed_image = transformed_image
    return transformed_image

#### treshold

In [ ]:
#tresholding
img_A = read_pgm_file('ImagenA.pgm')
show_img_hist(img_A)

print("Thresholding")
img_B_1 = img_A.copy()
thresholding(img_B_1)
show_img_hist(img_B_1,1,'informe_thresholding')

resta_img_1 = img_A.copy()
cv2.subtract(img_A, img_B_1, resta_img_1)
print("Diferencia entre Imagen A y Thresholding")
show_img_hist(resta_img_1,1,'informe_thresholding_diff')

#### gamma = 0.2

In [ ]:
#gamma 0.2
img_A = read_pgm_file('ImagenA.pgm')
show_img_hist(img_A)

print("Gamma Correction, gamma = 0.2")
img_B_2 = img_A.copy()
a = gamma_correction(img_B_2, 0.2)
show_img_hist(a, 1, 'informe_gamma_0.2')

resta_img_2 = img_A.copy()
resta_img_2 = img_A - a
print("Diferencia entre Imagen A y Gamma Correction, gamma = 0.2")
show_img_hist(resta_img_2, 1, 'informe_gamma_0.2_diff')

####    gamma = 3

In [ ]:
#gamma 2
img_A = read_pgm_file('ImagenA.pgm')
show_img_hist(img_A)

print("Gamma Correction, gamma = 3")
img_B_3 = img_A.copy()
b =gamma_correction(img_B_3, 3)
show_img_hist(b, 1, 'informe_gamma_3')

resta_img_3 = img_A.copy()
resta_img_3 = img_A - b
print("Diferencia entre Imagen A y Gamma Correction, gamma = 3")
show_img_hist(resta_img_3, 1, 'informe_gamma_3_diff')

##  Ex: 3 - REESCALEO

In [ ]:
img_C = read_pgm_file('ImagenC.pgm')

def dimention(img):
    H , W = img.shape
    print("Height: {}  Width: {}".format(H,W))
    return H,W


def Nearest_neighbor(img, scale):

    H , W = img.shape
    new_H = int(H*scale)
    new_W = int(W*scale)
    new_img = np.zeros((new_H,new_W),dtype=np.uint8)

    X_ratio = W/new_W
    Y_ratio = H/new_H

    for i in range(new_H):
        for j in range(new_W):
            x = int(j*X_ratio)
            y = int(i*Y_ratio)

            x_int = int(x)
            y_int = int(y)
            x_frac = x - x_int
            y_frac = y - y_int

            if x_frac >= 0.5:   x_int += 1
            if y_frac >= 0.5:   y_int += 1

            new_img[i,j] = img[y_int,x_int]

    return new_img

def Bilineal(img, scale, Cp =1):

    H , W = img.shape
    new_H = int(H*scale)
    new_W = int(W*scale)
    new_img = np.zeros((new_H,new_W),dtype=np.uint8)

    X_ratio = W/new_W
    Y_ratio = H/new_H

    for i in range(new_H):
        for j in range(new_W):
            x = j*X_ratio
            y = i*Y_ratio

            x_int = int(x)
            y_int = int(y)
            x_frac = x - x_int
            y_frac = y - y_int

            if x_int+1 < W and y_int+1 < H:
                if Cp == 1: 
                    C = (1-x_frac)*(1-y_frac)*img[y_int,x_int] + (x_frac)*(1-y_frac)*img[y_int,x_int+1] + (1-x_frac)*(y_frac)*img[y_int+1,x_int]+ (x_frac)*(y_frac)*img[y_int+1,x_int+1]
                else:    
                    C = img[y_int,x_int] + x_frac * (img[y_int,x_int+1]-img[y_int,x_int]) + y_frac * (img[y_int+1,x_int]-img[y_int,x_int]) + x_frac * y_frac * (img[y_int,x_int] + img[y_int+1,x_int+1]- img[y_int+1,x_int] - img[y_int,x_int+1])

                new_img[i,j] = C

            else:
                new_img[i,j] = img[y_int,x_int]

    return new_img



ratio = 1024/dimention(img_C)[0]


print("Original Image")
dimention(img_C)
show_img_hist(img_C,1,0,'informe_original')



print("Nearest Neighbor")
new_img = Nearest_neighbor(img_C,ratio)
dimention(new_img)
show_img_hist(new_img,1,0,'informe_NN')



print("Bilineal")
new_img = Bilineal(img_C,ratio,1)
dimention(new_img)
show_img_hist(new_img,1,0,'informe_Bilineal')
crop_bilineal = new_img[512:1024,512:1024]
show_img_hist(crop_bilineal,1,0,'informe_Bilineal_crop')


print('bilineal opencv')
new_img = cv2.resize(img_C, (1024,1024), interpolation = cv2.INTER_LINEAR)
dimention(new_img)
show_img_hist(new_img)


#interpolacion bicubica con openCV
print("Bicubic")
new_img = cv2.resize(img_C, (1024,1024), interpolation = cv2.INTER_CUBIC)
dimention(new_img)
show_img_hist(new_img,1,'informe_Bicubic')
crop_bicubic = new_img[512:1024,512:1024]
show_img_hist(crop_bicubic,1,0,'informe_Bicubic_crop')


##  Ex: 4 - FILTRADO

In [ ]:
# Low pass filter 
img_A = read_pgm_file('ImagenA.pgm')
img_C = read_pgm_file('ImagenC.pgm')

def Convolution_filter(img, kernel, tipe = 'valid'): #same, valid
    
    if tipe != 'valid' and tipe != 'same': 
        print("Invalid tipe")
        return
    
    if tipe == 'same': 
        img = np.pad(img,((2,2),(2,2)),'constant',constant_values=(0,0))

    H , W = img.shape
    new_img = np.zeros((H,W),dtype=np.float64)
    k_H , k_W = kernel.shape
    k_H = int(k_H/2)
    k_W = int(k_W/2)

    for i in range(H):
        for j in range(W):
            sum = 0
            for m in range(-k_H,k_H+1):
                for n in range(-k_W,k_W+1):
                    if i+m >= 0 and i+m < H and j+n >= 0 and j+n < W:
                        sum += img[i+m,j+n]*kernel[m+k_H,n+k_W]
            new_img[i,j] = sum
    return new_img

#kernel gaussiano
def gaussian_kernel(size, sigma):
    kernel = np.fromfunction(lambda x, y: (1/(2*np.pi*sigma**2)) * np.exp(-((x-(size-1)/2)**2 + (y-(size-1)/2)**2)/(2*sigma**2)), (size, size))
    return kernel/np.sum(kernel)

#convolution kernel opencv 
def Convolution_filter_opencv(img, kernel):
    new_img = cv2.filter2D(img, -1, kernel)
    return new_img


#imagen C
show_img_hist(img_C)

#Low pass filter 3*3
kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])/9
new_img = Convolution_filter(img_C,kernel)
print("Low pass filter 3*3")
show_img_hist(new_img,1,0,'informe_LPF_3x3')

#Low pass filter 5*5
kernel = np.array([[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]])/25
new_img = Convolution_filter(img_C,kernel)
print("Low pass filter 5*5")
show_img_hist(new_img,1,0,'informe_LPF_5x5')

#Low pass filter 7*7
kernel = np.array([[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1]])/49
new_img = Convolution_filter(img_C,kernel)
print("Low pass filter 7*7")
show_img_hist(new_img,1,0,'informe_LPF_7x7')

#imagen A
show_img_hist(img_A)

#Low pass filter 3*3
kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])/9
new_img = Convolution_filter(img_A,kernel)
print("Low pass filter 3*3")
show_img_hist(new_img,1,0,'informe_LPF_3x3_A')

#Low pass filter 5*5
kernel = np.array([[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]])/25
new_img = Convolution_filter(img_A,kernel)
print("Low pass filter 5*5")
show_img_hist(new_img,1,0,'informe_LPF_5x5_A')

#Low pass filter 7*7
kernel = np.array([[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1],[ 1, 1, 1, 1, 1, 1, 1]])/49
new_img = Convolution_filter(img_A,kernel)
print("Low pass filter 7*7")
show_img_hist(new_img,1,0,'informe_LPF_7x7_A')


# #High pass filter 5*5
# kernel = np.array([[-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1],[-1,-1,24,-1,-1],[-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1]])/25
# new_img = Convolution_filter(img_C,kernel)
# print("High pass filter 5*5")
# show_img_hist(new_img)
# print("High pass filter 5*5 opencv")
# new_img = Convolution_filter_opencv(img_C,kernel)
# show_img_hist(new_img)


## Ex: 5 - Fourier

In [ ]:
img_super = read_pgm_file('superman.pgm')
show_img_hist(img_super,1,0,'informe_superman')

fft_super = read_pgm_file('FFT of superman.pgm')
show_img_hist(fft_super,1,0,'informe_FFT_superman')

fft_super_cropped = read_pgm_file('FFT of superman_cropped.pgm')
show_img_hist(fft_super_cropped,1,0,'informe_FFT_superman_cropped')

invese_fft_super = read_pgm_file('Inverse FFT of superman_cropped.pgm')
show_img_hist(invese_fft_super,1,0,'informe_Inverse_FFT_superman_cropped')

## Ex: 6 - Unsharp Masking , highboost

In [ ]:
import scipy.signal as signal
img_A = read_pgm_file('ImagenA.pgm')

def gaussian_noise(img,mean,std, scale):
    noise = np.random.normal(mean, std, img.shape) * scale
    show_img_hist(noise)
    img = img + noise
    return img

def unsharp(img, kernel = 0):
    return hiboost(img, kernel, 1)

def hiboost(img, kernel = 0 , k=1):
    if kernel == 0: kernel = np.ones((3,3),np.float64)/9
    else : 
        if kernel % 2 == 0: 
            print("Invalid kernel")
            return
        kernel = np.ones((kernel,kernel),np.float64)/(kernel**2)
    img_1 = signal.convolve2d(img, kernel, mode='same')
    resta = img * k - img_1
    return resta

def Metrica(img):
    sum2 = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            sum2 += img[i,j]**2

    sum = np.sum(np.abs(img))
    return sum



print("Original Image")
#show_img_hist(img_A)

print("Gaussian Noise")
img_noise = gaussian_noise(img_A,0,1,10)
show_img_hist(img_noise, 1,1,'informe_gaussian_noise')

print("Unsharp")
img_noise_1 = img_noise.copy()
img_unsharp = unsharp(img_noise_1,3)
#show_img_hist(img_unsharp, 1,1,'informe_unsharp')

print("Hiboost")
img_noise_2 = img_noise.copy()
img_hiboost = hiboost(img_noise_2,3,2)
#show_img_hist(img_hiboost, 1,1,'informe_hiboost_k2')



M = []
X = np.linspace(0,10,100)
for i in X:
    #print("k: ",i,'\n\n')
    img_noise_2 = img_noise.copy()
    img_hiboost = hiboost(img_noise_2,0,i)
    #show_img_hist(img_hiboost)
    M.append(Metrica(img_A-img_hiboost))

plt.figure()
plt.grid()
plt.plot(X,M)
plt.xlabel('a')
plt.ylabel('Metrica')
plt.savefig('informe_metrica.png', bbox_inches='tight')

min = X[np.where(M == np.min(M))]
print("k minimo: ",min)
print("Metrica minima: ",np.min(M))
#show_img_hist(hiboost(img_noise_2,0,min))

## Ex: 7 - CT HS

### a) 

In [ ]:
img_02 = read_pgm_file('AAA0002.pgm')
img_03 = read_pgm_file('AAA0003.pgm')
img_04 = read_pgm_file('AAA0004.pgm')

#dibujar un cuadrado en la imagen que representa la roi a tomar
new_img = img_02.copy()
cv2.rectangle(new_img, (130,170), (400,350), (255,255,255), 3)
show_img_hist(new_img,1,0,'informe_02_roi')
new_img = img_03.copy()
cv2.rectangle(new_img, (130,170), (400,350), (255,255,255), 3)
show_img_hist(new_img,1,0,'informe_03_roi')
new_img = img_04.copy()
cv2.rectangle(new_img, (130,170), (400,350), (255,255,255), 3)
show_img_hist(new_img,1,0,'informe_04_roi')


#crop img fila 170 -> 350 ; columna 130 -> 400
img_02 = img_02[170:350,130:400]
img_03 = img_03[170:350,130:400]
img_04 = img_04[170:350,130:400]

show_img_hist(img_02,0,1,'informe_02_crop')
show_img_hist(img_03,0,1,'informe_03_crop')
show_img_hist(img_04,0,1,'informe_04_crop')

def SN_relation(img):
    mean = np.mean(img)
    std = np.std(img)
    return mean/std

print("SN relation img_02: ",SN_relation(img_02))
print("SN relation img_03: ",SN_relation(img_03))
print("SN relation img_04: ",SN_relation(img_04))


### b)

In [ ]:
img_11 = read_pgm_file('AAA0011.pgm')
img_12 = read_pgm_file('AAA0012.pgm')
img_13 = read_pgm_file('AAA0013.pgm')
img_14 = read_pgm_file('AAA0014.pgm')

new_img = img_11.copy()
cv2.rectangle(new_img, (200,147-20), (240,148+20), (255,255,255), 2)
show_img_hist(new_img,1,0,'informe_11_roi')
new_img = new_img[147-20:148+20,200:240]
cv2.line(new_img, (0,20), (40,20), (255,255,255), 1)
show_img_hist(new_img,1,0,'informe_11_roi_crop')

new_img = img_12.copy()
cv2.rectangle(new_img, (200,147-20), (240,148+20), (255,255,255), 2)
show_img_hist(new_img,1,0,'informe_12_roi')
new_img = new_img[147-20:148+20,200:240]
cv2.line(new_img, (0,20), (40,20), (255,255,255), 1)
show_img_hist(new_img,1,0,'informe_12_roi_crop')

new_img = img_13.copy()
cv2.rectangle(new_img, (200,147-20), (240,148+20), (255,255,255), 2)
show_img_hist(new_img,1,0,'informe_13_roi')
new_img = new_img[147-20:148+20,200:240]
cv2.line(new_img, (0,20), (40,20), (255,255,255), 1)
show_img_hist(new_img,1,0,'informe_13_roi_crop')

new_img = img_14.copy()
cv2.rectangle(new_img, (200,147-20), (240,148+20), (255,255,255), 2)
show_img_hist(new_img,1,0,'informe_14_roi')
new_img = new_img[147-20:148+20,200:240]
cv2.line(new_img, (0,20), (40,20), (255,255,255), 1)
show_img_hist(new_img,1,0,'informe_14_roi_crop')
plt.show()

#me quedo solo con la fila 147
Fila = 148
H_11= img_11[Fila,:]
H_12= img_12[Fila,:]
H_13= img_13[Fila,:]
H_14= img_14[Fila,:]
Col_1 = 200
Col_2 = 240
H_11 = H_11[Col_1:Col_2]
H_12 = H_12[Col_1:Col_2]
H_13 = H_13[Col_1:Col_2]
H_14 = H_14[Col_1:Col_2]



#busco el FWHM
def FWHM(H,num):
    plt.grid()
    plt.plot(np.linspace(Col_1,Col_2,Col_2-Col_1) ,H, marker='o')
    H_max = np.max(H)
    H_med = H_max/2
    plt.hlines(H_med,Col_1,Col_2,linestyles='dashed', colors='r')
    plt.xlabel('Columna')
    plt.ylabel('Intensidad')
    plt.title('Perfil de intensidad')
    H_max = np.where(H == H_max)
    H_med = np.where(H >= H_med)
    FWHM = H_med[0][-1] - H_med[0][0]
    print("FWHM: ",FWHM, "Max: ",H_max[0][0], "Med: ",H_med[0][0], H_med[0][-1])
    plt.savefig('informe_FWHM_'+str(num)+'.png', bbox_inches='tight')
    plt.show()
    return FWHM, np.max(H)/2

a11 =FWHM(H_11,11)
a12 =FWHM(H_12,12)
a13 =FWHM(H_13,13)
a14 =FWHM(H_14,14)

print("FWHM STD: ",a11)
print("FWHM EDGE: ",a12)
print("FWHM SOFT: ",a13)
print("FWHM CHST: ",a14)